In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import models
from torch.nn import functional as F
from torchvision import transforms as T
from tqdm.notebook import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using device {device}")

using device cpu


In [ ]:
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer_1a = nn.Linear(10, 200)
    self.layer_1b = nn.Linear(200, 2000)
    self.model = nn.Sequential(
        nn.Linear(2200, 2200),
        nn.ReLU(),
        nn.Linear(2200, 2200),
        nn.ReLU(),
        nn.Linear(2200, 2200),
        nn.ReLU(),
        nn.Linear(2200, 28*28),
        nn.Tanh(),
    )

  def forward(self, x):
    layer1a = F.relu(self.layer_1a(x[:, :10]))
    layer1b = F.relu(self.layer_1b(x[:, 10:]))
    x = torch.cat((layer1a, layer1b), dim=1)
    x = self.model(x)
    return x.view(x.shape[0], 28, 28)

In [ ]:
generator = Generator()

In [ ]:
discriminator = models.resnet18()
discriminator.conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [ ]:
discriminator.conv1 = nn.Conv2d(1, 64, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)

In [ ]:
discriminator.fc

Linear(in_features=512, out_features=1000, bias=True)

In [ ]:
discriminator.fc.in_features

512

In [ ]:
discriminator.fc = nn.Sequential(
    nn.Linear(discriminator.fc.in_features, 10),
    nn.Sigmoid(),)

In [ ]:
d_optimizer = torch.optim.Adam(params=discriminator.parameters(), lr=.001, betas=(.5, .999))

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
transform = T.Compose([
  T.ToTensor(),
  T.Resize(size=(224, 224)),
])
train_data = torchvision.datasets.MNIST(root='.', train=True, download=True, transform=transform)
test_data = torchvision.datasets.MNIST(root='.', train=False, download=True, transform=transform)

In [ ]:
type(test_data)

torchvision.datasets.mnist.MNIST

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

In [ ]:
def train(model, loader, optimizer, loss_fn, device):
  running_loss = 0
  count = len(loader)
  for data, labels in tqdm(loader, total=count, desc="training"):
    model.zero_grad()
    labels = labels.to(device)
    data = data.to(device)
    out = model(data)
    loss = loss_fn(out, labels)
    loss.backward()
    running_loss += loss.item()

  avg_loss = running_loss / len(loader)
  print(f"training loss: {avg_loss}")
  return avg_loss

In [ ]:
epochs = 2
training_losses = []
discriminator = discriminator.to(device)
for epoch in range(epochs):
  training_loss = train(discriminator, train_loader, d_optimizer, loss_fn, device)

training:   0%|          | 0/600 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
for batch in train_loader:
  break

In [ ]:
labels, data = batch

In [ ]:
data